In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Randomly seeding
tf.random.set_seed(6950)

# Biosppy for signal processing
from biosppy.signals import ecg

# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}


# Feature extraction function
def extract_comprehensive_features(ecg_signal):
    """
    Extract comprehensive features from entire ECG signal
    """
    try:
        ecg_processed = ecg.ecg(signal=ecg_signal, sampling_rate=SAMPLING_RATE, show=False)
        rpeaks = ecg_processed['rpeaks']
        heart_rate = ecg_processed['heart_rate']

        features = [
            np.mean(ecg_signal),
            np.std(ecg_signal),
            np.median(ecg_signal),
            np.min(ecg_signal),
            np.max(ecg_signal),
            len(rpeaks),
            np.mean(heart_rate) if len(heart_rate) > 0 else 0,
            np.std(heart_rate) if len(heart_rate) > 0 else 0,
            np.mean(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.std(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.percentile(ecg_signal, 25),
            np.percentile(ecg_signal, 75),
        ]

        return features

    except Exception as e:
        print(f"Error processing signal: {e}")
        return [0] * 12


# Dataset preparation function
def prepare_dataset(ecg_folder, diagnostics_df):
    signals, signal_features, signal_labels = [], [], []

    for _, row in diagnostics_df.iterrows():
        file_name = row['FileName']
        rhythm_label = row['Rhythm']

        if pd.isnull(rhythm_label) or rhythm_label not in rhythm_mapping.values():
            continue

        ecg_file = os.path.join(ecg_folder, f"{file_name}.csv")
        if not os.path.exists(ecg_file):
            continue

        ecg_data = pd.read_csv(ecg_file, header=0).iloc[:, 1].values
        ecg_data = ecg_data.astype(float)

        features = extract_comprehensive_features(ecg_data)
        signals.append(ecg_data)
        signal_features.append(features)
        signal_labels.append(rhythm_label)

    return np.array(signals), np.array(signal_features), np.array(signal_labels)


# Load and process data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)

raw_signals, signal_features, signal_labels = prepare_dataset(ecg_folder, diagnostics_df)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(signal_labels)
onehot_labels = to_categorical(encoded_labels)

feature_scaler = StandardScaler()
scaled_features = feature_scaler.fit_transform(signal_features)


# Create MLP model
def create_mlp_model(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model


# Stratified K-Fold Cross-Validation with optimizer tuning
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
results = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for optimizer in optimizers:
    print(f"\nTesting optimizer: {optimizer}")
    cv_scores = []

    for train_index, test_index in skf.split(scaled_features, encoded_labels):
        X_train, X_test = scaled_features[train_index], scaled_features[test_index]
        y_train, y_test = onehot_labels[train_index], onehot_labels[test_index]

        model = create_mlp_model(input_shape=X_train.shape[1], num_classes=onehot_labels.shape[1])
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        cv_scores.append(accuracy)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

    results[optimizer] = np.mean(cv_scores)
    print(f"Optimizer: {optimizer} - Mean CV Accuracy: {np.mean(cv_scores)}")

print("\nFinal Results:")
for opt, acc in results.items():
    print(f"{opt}: {acc}")

2024-11-29 09:18:58.928056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 09:18:58.938969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 09:18:58.942516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 09:18:58.951644: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 09:18:59.483678: W tensorflow/compiler/tf2


Testing optimizer: adam


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732850624.156794  543763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850624.189516  543763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850624.

Epoch 1/100


d negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850624.202110  543763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850624.303141  543763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850624.304300  543763 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See mo

...

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2625 - loss: 1.3867 - val_accuracy: 0.3580 - val_loss: 1.3455
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.4890 - loss: 1.2735 - val_accuracy: 0.4155 - val_loss: 1.2911
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.5850 - loss: 1.1918 - val_accuracy: 0.4225 - val_loss: 1.2420
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.6263 - loss: 1.1142 - val_accuracy: 0.4296 - val_loss: 1.1962
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.6337 - loss: 1.0488 - val_accuracy: 0.4501 - val_loss: 1.1533
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - accuracy: 0.6498 - loss: 0.9872 - val_accuracy: 0.4683 - val_loss: 1.1119
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.6669 - loss: 0.9420 - val_accuracy: 0.4930 - val_loss: 1.0749
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.6778 - loss: 0.

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.2652 - loss: 1.3804 - val_accuracy: 0.4202 - val_loss: 1.3289
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.4969 - loss: 1.2901 - val_accuracy: 0.3926 - val_loss: 1.3052
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - accuracy: 0.5591 - loss: 1.2204 - val_accuracy: 0.3862 - val_loss: 1.2817
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.5871 - loss: 1.1586 - val_accuracy: 0.3891 - val_loss: 1.2585
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.5946 - loss: 1.1001 - val_accuracy: 0.3903 - val_loss: 1.2368
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.6045 - loss: 1.0470 - val_accuracy: 0.4026 - val_loss: 1.2120
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step - accuracy: 0.6025 - loss: 1.0070 - val_accuracy: 0.4219 - val_loss: 1.1846
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.6219 - loss: 0.9630 - val_a

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2518 - loss: 1.3647 - val_accuracy: 0.3351 - val_loss: 1.3548
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.3859 - loss: 1.3160 - val_accuracy: 0.5252 - val_loss: 1.3020
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.5267 - loss: 1.2661 - val_accuracy: 0.5851 - val_loss: 1.2495
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step - accuracy: 0.6077 - loss: 1.2153 - val_accuracy: 0.6174 - val_loss: 1.1984
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.6250 - loss: 1.1574 - val_accuracy: 0.6373 - val_loss: 1.1480
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.6466 - loss: 1.1026 - val_accuracy: 0.6538 - val_loss: 1.0969
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.6577 - loss: 1.0436 - val_accuracy: 0.6708 - val_loss: 1.0485
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.6731 - loss: 0.9901 - val_a